In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]

# Validate a Model Post Run

After executing a model, there may be an interest to check the values of certain inputs or outputs. Although the values may be mathematically converged and correct, the inputs may have been set resulting in an output value that you do not want or that is not physical. If an input / output is found to have an undesired value, this may prompt you to change the value of an input, discretely change the structure and/or connections of the model, or raise a warning if a variable is close to an undesired bound.

Once `run_model()` or `final_setup()` is finished the `run_validation()` method may be run. This method can be run by any system in the model. It will go down the model hierarchy relative to the system that ran the `run_validation()`, and run the `validate` method on each system in the hierarchy. The `validate` method on a system takes in the inputs / outputs in a read-only mode and can be overwritten to do whatever post-run checks are necessary in the component.

```{eval-rst}
    .. automethod:: openmdao.core.system.System.validate
        :noindex:
```

## Examples

Set up a simple problem and run it, afterwards calling `run_validation`.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src50", get_code("openmdao.test_suite.components.sellar.SellarDis1"), display=False)
glue("code_src50", get_code("openmdao.test_suite.components.sellar.SellarDis2"), display=False)

:::{Admonition} `SellarDis1` class definition 
:class: dropdown

{glue:}`code_src50`
:::

:::{Admonition} `SellarDis2` class definition 
:class: dropdown

{glue:}`code_src50`
:::

In [ ]:
import warnings
import numpy as np
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDis1, SellarDis2

class ValidatedSellar1(SellarDis1):
    def validate(self, inputs, outputs):
        if outputs['y1'] > 20.0:
            warnings.warn(f'Output "y1" in component "{self.name}" is greater than 20.')


class SellarMDA(om.Group):
    def setup(self):
        cycle = self.add_subsystem('cycle', om.Group(), promotes=['*'])
        cycle.add_subsystem('d1', ValidatedSellar1(), promotes_inputs=['x', 'z', 'y2'],
                            promotes_outputs=['y1'])
        cycle.add_subsystem('d2', SellarDis2(), promotes_inputs=['z', 'y1'],
                            promotes_outputs=['y2'])

        cycle.set_input_defaults('x', 1.0)
        cycle.set_input_defaults('z', np.array([5.0, 2.0]))

        # Nonlinear Block Gauss Seidel is a gradient free solver
        cycle.nonlinear_solver = om.NonlinearBlockGS()

prob = om.Problem(model=om.Group())
prob.model.add_subsystem('cycle', SellarMDA())
prob.setup()
prob.run_model()
prob.model.run_validation()